In [1]:
import pandas as pd
import math
from itertools import combinations

def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Earth radius in km
    dLat = math.radians(lat2 - lat1)
    dLon = math.radians(lon2 - lon1)
    a = math.sin(dLat/2)**2 + math.cos(math.radians(lat1)) * math.cos(math.radians(lat2)) * math.sin(dLon/2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    return R * c

def compute_mst(points):
    n = len(points)
    edges = []
    for i in range(n):
        for j in range(i + 1, n):
            dist = haversine(points[i][0], points[i][1], points[j][0], points[j][1])
            edges.append((dist, i, j))
    edges.sort()
    parent = list(range(n))
    def find(u):
        while parent[u] != u:
            parent[u] = parent[parent[u]]
            u = parent[u]
        return u
    def union(u, v):
        u_root = find(u)
        v_root = find(v)
        if u_root != v_root:
            parent[v_root] = u_root
    mst_sum = 0
    mst_edges = []
    for edge in edges:
        dist, u, v = edge
        if find(u) != find(v):
            union(u, v)
            mst_sum += dist
            mst_edges.append(edge)
            if len(mst_edges) == n - 1:
                break
    return mst_sum

def convert_time_to_minutes(time_str):
    return int(time_str.split(':')[0]) * 60 + int(time_str.split(':')[1])

def tsp_nearest_neighbor(points):
    n = len(points)
    visited = [False] * n
    path = [0]
    visited[0] = True
    current = 0
    for _ in range(n - 1):
        nearest_dist = float('inf')
        nearest_idx = -1
        for i in range(n):
            if not visited[i]:
                dist = haversine(points[current][0], points[current][1], points[i][0], points[i][1])
                if dist < nearest_dist:
                    nearest_dist = dist
                    nearest_idx = i
        if nearest_idx == -1:
            break
        path.append(nearest_idx)
        visited[nearest_idx] = True
        current = nearest_idx
    path.append(0)
    return path

def check_capacity_constraints(num_shipments, vehicle_capacity):
    # Check if shipments are between 50% and 100% of vehicle capacity
    min_shipments = math.ceil(vehicle_capacity * 0.5)
    max_shipments = vehicle_capacity
    return min_shipments <= num_shipments <= max_shipments

# Load data
store_df = pd.read_excel('Data/SmartRoute Optimizer.xlsx', sheet_name='Store Location')
store_lat = store_df['Latitute']
store_lon = store_df['Longitude']

shipments_df = pd.read_excel('Data/SmartRoute Optimizer.xlsx', sheet_name='Shipments_Data')
shipments = []
for _, row in shipments_df.iterrows():
    start, end = row['Delivery Timeslot'].split('-')
    shipments.append({
        'id': row['Shipment ID'],
        'lat': row['Latitude'],
        'lon': row['Longitude'],
        'start': convert_time_to_minutes(start.strip()),
        'end': convert_time_to_minutes(end.strip())
    })

# Priority vehicles (3W, 4W-EV) and regular 4W
vehicles = [
    {'type': '3W', 'remaining': 50, 'capacity': 5, 'max_radius': 15},
    {'type': '4W-EV', 'remaining': 25, 'capacity': 8, 'max_radius': 20},
    {'type': '4W', 'remaining': float('inf'), 'capacity': 25, 'max_radius': float('inf')}
]

# Sort shipments by start time and group them by timeslot
shipments.sort(key=lambda x: x['start'])
timeslot_groups = {}
for shipment in shipments:
    key = (shipment['start'], shipment['end'])
    if key not in timeslot_groups:
        timeslot_groups[key] = []
    timeslot_groups[key].append(shipment)

trips = []

# Process each timeslot group
for (start_time, end_time), group_shipments in timeslot_groups.items():
    current_batch = []
    
    # Process shipments in the current timeslot
    for shipment in group_shipments:
        current_batch.append(shipment)
        
        # Try to create a trip when we have enough shipments
        for vehicle in vehicles:
            if vehicle['remaining'] <= 0:
                continue
                
            if check_capacity_constraints(len(current_batch), vehicle['capacity']):
                points = [(store_lat, store_lon)] + [(s['lat'], s['lon']) for s in current_batch]
                mst_dist = compute_mst(points)
                
                if mst_dist <= vehicle['max_radius'] or vehicle['type'] == '4W':
                    trip_time = (mst_dist * 5) + (len(current_batch) * 10)
                    available_time = end_time - start_time
                    
                    if trip_time <= available_time:
                        trips.append({
                            'shipments': current_batch.copy(),
                            'start': start_time,
                            'end': end_time,
                            'mst_dist': mst_dist,
                            'vehicle': vehicle['type'],
                            'vehicle_capacity': vehicle['capacity'],
                            'vehicle_max_radius': vehicle['max_radius']
                        })
                        vehicle['remaining'] -= 1
                        current_batch = []
                        break
    
    # Handle remaining shipments in the batch
    if current_batch:
        # Try to find the best fitting vehicle for remaining shipments
        for vehicle in vehicles:
            if vehicle['remaining'] <= 0:
                continue
            
            if len(current_batch) <= vehicle['capacity']:
                points = [(store_lat, store_lon)] + [(s['lat'], s['lon']) for s in current_batch]
                mst_dist = compute_mst(points)
                
                if mst_dist <= vehicle['max_radius'] or vehicle['type'] == '4W':
                    trips.append({
                        'shipments': current_batch.copy(),
                        'start': start_time,
                        'end': end_time,
                        'mst_dist': mst_dist,
                        'vehicle': vehicle['type'],
                        'vehicle_capacity': vehicle['capacity'],
                        'vehicle_max_radius': vehicle['max_radius']
                    })
                    vehicle['remaining'] -= 1
                    current_batch = []
                    break

# Sequence shipments in each trip using TSP
for trip in trips:
    points = [(store_lat, store_lon)] + [(s['lat'], s['lon']) for s in trip['shipments']]
    path = tsp_nearest_neighbor(points)
    shipment_indices = path[1:-1]  # exclude store at start and end
    ordered_shipments = [trip['shipments'][i-1] for i in shipment_indices]
    trip['shipments'] = ordered_shipments

# Generate output
output = []
for idx, trip in enumerate(trips):
    trip_time = (trip['mst_dist'] * 5) + (len(trip['shipments']) * 10)
    available_time = trip['end'] - trip['start']
    time_uti = trip_time / available_time if available_time != 0 else 0
    capacity_uti = len(trip['shipments']) / trip['vehicle_capacity']
    cov_uti = trip['mst_dist'] / trip['vehicle_max_radius'] if trip['vehicle_max_radius'] != float('inf') else 0
    
    output.append({
        'Trip_ID': idx + 1,
        'Shipment_Sequence': [s['id'] for s in trip['shipments']],
        'Vehicle_Type': trip['vehicle'],
        'MST_DIST': round(trip['mst_dist'], 2),
        'TRIP_TIME': round(trip_time, 2),
        'CAPACITY_UTI': round(capacity_uti, 2),
        'TIME_UTI': round(time_uti, 2),
        'COV_UTI': round(cov_uti, 2) if isinstance(cov_uti, float) else 'N/A'
    })

# Convert to DataFrame
output_df = pd.DataFrame(output)
print(output_df.to_string(index=False))

 Trip_ID    Shipment_Sequence Vehicle_Type  MST_DIST  TRIP_TIME  CAPACITY_UTI  TIME_UTI  COV_UTI
       1            [3, 8, 7]           3W     12.91      94.57           0.6      0.63     0.86
       2     [19, 24, 25, 23]        4W-EV     17.41     127.07           0.5      0.85     0.87
       3            [1, 2, 5]           3W     12.84      94.22           0.6      0.63     0.86
       4       [9, 6, 11, 10]        4W-EV     17.83     129.13           0.5      0.86     0.89
       5      [68, 53, 70, 4]        4W-EV     16.27     121.34           0.5      0.81     0.81
       6     [97, 91, 74, 72]        4W-EV     17.02     125.08           0.5      0.83     0.85
       7      [120, 107, 110]           3W     13.62      98.12           0.6      0.65     0.91
       8      [177, 138, 179]           3W     13.49      97.47           0.6      0.65     0.90
       9      [232, 199, 207]           3W      9.18      75.92           0.6      0.51     0.61
      10      [270, 247, 237] 

/tmp/ipykernel_34548/3541125056.py:7: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  dLat = math.radians(lat2 - lat1)
/tmp/ipykernel_34548/3541125056.py:8: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  dLon = math.radians(lon2 - lon1)
/tmp/ipykernel_34548/3541125056.py:9: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  a = math.sin(dLat/2)**2 + math.cos(math.radians(lat1)) * math.cos(math.radians(lat2)) * math.sin(dLon/2)**2
